#Part One - Scrape the list of US presidents from https://en.wikipedia.org/wiki/List_of_presidents_of_the_United_States. 
using pandas and save them as a CSV.

In [3]:
import pandas as pd

In [4]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [5]:
tables = pd.read_html('https://en.wikipedia.org/wiki/List_of_presidents_of_the_United_States')
print(tables)

[    No.[a]  Portrait                       Name (birth–death)  \
0        1       NaN       George Washington (1732–1799) [19]   
1        2       NaN              John Adams (1735–1826) [21]   
2        3       NaN        Thomas Jefferson (1743–1826) [23]   
3        4       NaN           James Madison (1751–1836) [24]   
4        5       NaN            James Monroe (1758–1831) [26]   
5        6       NaN       John Quincy Adams (1767–1848) [27]   
6        7       NaN          Andrew Jackson (1767–1845) [30]   
7        8       NaN        Martin Van Buren (1782–1862) [31]   
8        9       NaN  William Henry Harrison (1773–1841) [32]   
9       10       NaN              John Tyler (1790–1862) [33]   
10      11       NaN           James K. Polk (1795–1849) [36]   
11      12       NaN          Zachary Taylor (1784–1850) [37]   
12      13       NaN        Millard Fillmore (1800–1874) [38]   
13      14       NaN         Franklin Pierce (1804–1869) [40]   
14      15       NaN    

In [6]:
tables[0]

,No.[a],Portrait,Name (birth–death),Term[16],Party[b][17],Party[b][17].1,Election,Vice President[18]
0,1,NaN,George Washington (1732–1799) [19],"April 30, 1789 – March 4, 1797",NaN,Unaffiliated,1788–891792,John Adams[c]
1,2,NaN,John Adams (1735–1826) [21],"March 4, 1797 – March 4, 1801",NaN,Federalist,1796,Thomas Jefferson[d]
2,3,NaN,Thomas Jefferson (1743–1826) [23],"March 4, 1801 – March 4, 1809",NaN,Democratic- Republican,1800 1804,Aaron BurrGeorge Clinton
3,4,NaN,James Madison (1751–1836) [24],"March 4, 1809 – March 4, 1817",NaN,Democratic- Republican,18081812,"George Clinton[e]Vacant after April 20, 1812El..."
4,5,NaN,James Monroe (1758–1831) [26],"March 4, 1817 – March 4, 1825",NaN,Democratic- Republican,18161820,Daniel D. Tompkins
5,6,NaN,John Quincy Adams (1767–1848) [27],"March 4, 1825 – March 4, 1829",NaN,Democratic- Republican[f]National Republican,1824,John C. Calhoun[g]
6,7,NaN,Andrew Jackson (1767–1845) [30],"March 4, 1829 – March 4, 1837",NaN,Democratic,18281832,"John C. Calhoun[h]Vacant after December 28, 18..."
7,8,NaN,Martin Van Buren (1782–1862) [31],"March 4, 1837 – March 4, 1841",NaN,Democratic,1836,Richard Mentor Johnson
8,9,NaN,William Henry Harrison (1773–1841) [32],"March 4, 1841 – April 4, 1841[e]",NaN,Whig,1840,John Tyler
9,10,NaN,John Tyler (1790–1862) [33],"April 4, 1841[i] – March 4, 1845",NaN,Whig[j]Unaffiliated,–,Vacant throughout presidency


In [7]:
tables[0].to_csv('presidents.csv', index = False)

PART TWO - Scrape the content of https://www.lemonde.fr/Links to an external site. and save it as a CSV.

We want: titles, subhead, article URL, whether it's premium or not, byline, article type, image URL.

You're going to be missing some article URLs - you should use try/except on them to avoid them being a problem, buuuut after that I would recommend printing the element to see why your approach isn't working (it's 100% possible to get all of the URLs!)
If you want to make sure the element you're getting has a specific attribute, you can do something like .select_one("a[href]"). That will only give you anchor tags (links) that have hrefs.
Instead of yes/no for the premium question, you can think of it as "put text in this column if the article is premium, don't put text in it if it is not"

In [54]:
from bs4 import BeautifulSoup
import requests

response = requests.get("https://www.lemonde.fr/en/")
doc = BeautifulSoup(response.text)

In [89]:
items = doc.find_all(class_='article')

In [125]:
rows = []
for item in items:
    row = {}
    row['title'] = item.find(class_='article__title').text
    try:
        row['url'] = item.find(class_="article article--nav").get('href')
    except:
        row['url'] = item.get('href', None)
    try:
        row['premium'] = item.find(class_='icon__premium').text
    except:
        pass
    try:
        row['image url'] = item.find(class_='lmd-link-clickarea__link').get('href')
    except:
        row['image url'] = item.get('href', None)
    try:
        row['subhed'] = item.find(class_="article__desc").text
    except:
        pass
    try:
        row['article__type'] = item.find(class_='article__type').text
    except: 
        pass
    try:
        row['article__byline'] = item.find(class_='article__type').text
    except: 
        pass
    rows.append(row)
rows

[{'title': 'Israel thwarts Iranian retaliation and continues military offensive',
  'url': 'https://www.lemonde.fr/en/international/article/2025/06/14/israel-thwarts-iranian-retaliation-and-continues-military-offensive_6742346_4.html',
  'premium': ' Subscribers only ',
  'image url': 'https://www.lemonde.fr/en/international/article/2025/06/14/israel-thwarts-iranian-retaliation-and-continues-military-offensive_6742346_4.html'},
 {'title': "The collapse of the 'axis of resistance'",
  'url': 'https://www.lemonde.fr/en/international/article/2025/06/14/israel-s-attack-on-iran-the-collapse-of-the-pro-tehran-axis-of-resistance_6742345_4.html',
  'premium': ' Subscribers only ',
  'image url': 'https://www.lemonde.fr/en/international/article/2025/06/14/israel-s-attack-on-iran-the-collapse-of-the-pro-tehran-axis-of-resistance_6742345_4.html'},
 {'title': "Attack on Iran derails France's plans to recognize a Palestinian state",
  'url': 'https://www.lemonde.fr/en/international/article/2025/06/

In [126]:
import pandas as pd
df = pd.json_normalize(rows)
df.head()
df.to_csv("lemonde.csv", index=False)